In [ ]:
import numpy as np
import pandas as pd
from copy import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn import metrics

In [ ]:
file_path ="C:/Users/Andrew Mark/Google Drive/Projects/Kaggle/house prices"
traindf = pd.read_csv(file_path+"/train.csv")
testdf = pd.read_csv(file_path+"/test.csv")
subdf = pd.read_csv(file_path+"/sample_submission.csv")

train = copy(traindf)
test = copy(testdf)
submission = copy(subdf)

In [ ]:
# add the SalePrice column to the test dataset
test['SalePrice'] = submission['SalePrice'].values

In [271]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 81 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1459 non-null int64
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null int64
Alley            1459 non-null int64
LotShape         1459 non-null int64
LandContour      1459 non-null int64
Utilities        1459 non-null int64
LotConfig        1459 non-null int64
LandSlope        1459 non-null int64
Neighborhood     1459 non-null int64
Condition1       1459 non-null int64
Condition2       1459 non-null int64
BldgType         1459 non-null int64
HouseStyle       1459 non-null int64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null int64
RoofMatl         1459 non-null int64
Exterior1st      1459 non-null int64
Ext

In [ ]:
#Filling missing values
def fill_missing_values(df):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    for column in list(missing.index):
        if df[column].dtype == 'object':
            df[column].fillna(df[column].value_counts().index[0], inplace=True)
        elif df[column].dtype == 'int64' or 'float64' or 'int16' or 'float16':
            df[column].fillna(df[column].median(), inplace=True)          

In [ ]:
#Converting String to Int
def convert (df):
    # Find the columns of object type along with their column index
    object_cols = list(df.select_dtypes(exclude=[np.number]).columns)
    object_cols_ind = []
    for col in object_cols:
        object_cols_ind.append(df.columns.get_loc(col))

    # Encode the categorical columns with numbers    
    label_enc = LabelEncoder()
    for i in object_cols_ind:
        df.iloc[:,i] = label_enc.fit_transform(df.iloc[:,i])

In [ ]:
fill_missing_values(train)
fill_missing_values(test)

In [ ]:
convert(train)
convert(test)

In [ ]:
print("Train Dtype counts: \n{}".format(train.dtypes.value_counts()))
print("Test Dtype counts: \n{}".format(test.dtypes.value_counts()))

In [ ]:
train = train.drop(train.index[47])

In [264]:
#Machine Learning

X_train = train.drop(['SalePrice','Id'], axis=1)
y_train = train['SalePrice']

X_test = test.drop(['SalePrice','Id'], axis=1)
y_test = test['SalePrice']

In [265]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
y_test = lab_enc.fit_transform(y_test)

In [269]:
# training and prediction
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB

logmodel = LogisticRegression()
logmodel.fit(X_train,y_test)
lr_pred = logmodel.predict(X_test)

rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_test)
rf_pred = rf.predict(X_test)

gnb = GaussianNB()
gnb.fit(X_train, y_test)
gnb_pred = gnb.predict(X_test)

g_boost = GradientBoostingRegressor( n_estimators=6000, learning_rate=0.01,max_depth=5, max_features='sqrt',
                                    min_samples_leaf=15, min_samples_split=10, loss='ls', random_state =101)
g_boost.fit(X_train,y_test)
gbm_pred = g_boost.predict(X_test)

In [268]:
print("Logistic Regression RMSE: \t",metrics.mean_squared_error(y_test, lr_pred))
print("Random Forest RMSE: \t\t", metrics.mean_squared_error(y_test, rf_pred))
print("Naive Bayes RMSE: \t\t", metrics.mean_squared_error(y_test, gnb_pred))
print("Gradient Boosting RMSE: \t", metrics.mean_squared_error(y_test, gnb_pred))
print("\n")

Logistic Regression RMSE: 	 35648877686.686775
Random Forest RMSE: 		 417467.11857436603
Naive Bayes RMSE: 		 215462.15147361206
Gradient Boosting RMSE: 	 215462.15147361206




In [ ]:
submission = pd.DataFrame()
submission['Id'] = np.array(test.index)
submission['SalePrice'] = prediction
submission.to_csv('house.csv', index=False)